# INISIALISASI TRAINING

In [ ]:
from ultralytics import YOLO

In [ ]:
!yolo task=detect mode=predict model=../.runs/weights/yolov8l.pt conf=0.25 source="D:/NWR27/MY_FILES/Datasets/z_arc/Murtaza/Images/People.jpg" project="../.runs/detect/.try" name="predict"

In [ ]:
import torch

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

In [ ]:
from IPython.display import display
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

# STARTING TRAINING WITH REALTIME OUTPUT

In [ ]:
import subprocess
import sys
import threading


def read_output(pipe, pipe_name):
    for line in iter(pipe.readline, ""):
        if pipe_name == "stderr" and "ERROR" in line:
            print(f"ERROR: {line}", end="", file=sys.stderr)
        else:
            if "Epoch" in line:
                print()  # Print a newline before printing epoch info
            print(line, end="\r" if "Epoch" not in line else "\n")
        sys.stdout.flush()
    pipe.close()


command = [
    "yolo",
    "task=detect",
    "mode=train",
    "model=../.runs/weights/yolov8l.pt",
    "data=D:/NWR27/MY_FILES/Datasets/employees_3_original++/data.yaml",
    "epochs=50",
    "imgsz=640",
    "project=../.runs/detect",
    "name=employees_3_original++",
]

process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, bufsize=1)

# Membuat thread untuk menangani stdout dan stderr
stdout_thread = threading.Thread(target=read_output, args=(process.stdout, "stdout"))
stderr_thread = threading.Thread(target=read_output, args=(process.stderr, "stderr"))

# Memulai thread
stdout_thread.start()
stderr_thread.start()

# Menunggu proses selesai
process.wait()

# Menunggu thread selesai
stdout_thread.join()
stderr_thread.join()